In [21]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [22]:
directory = 'MlDump'
pd.options.mode.chained_assignment = None   # !!!!!!!!!!!!! отключает вывод предупреждений pandas

In [23]:
DS = pd.DataFrame()

In [24]:
# Конкатенация всех дампов в один датафрейм
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ds = pd.read_csv(f)
        DS = pd.concat([DS, ds])

In [25]:
# Сброс индекса
DS.reset_index(drop= True , inplace= True )

In [26]:
# Очистка датасета от лишних данных (убираем столбец со всеми значениями NaN и столбец с нумерацией строк)
del DS["bInterfaceClass"]
del ds["No."]

In [27]:
# Кодирование столбцов с текстовыми значениями
le_Source = LabelEncoder()
DS['Source_new'] = le_Source.fit_transform(DS.Source.values)
le_Destination = LabelEncoder()
DS['Destination_new'] = le_Destination.fit_transform(DS.Destination.values)
le_Protocol = LabelEncoder()
DS['Protocol_new'] = le_Protocol.fit_transform(DS.Protocol.values)
le_Info = LabelEncoder()
DS['Info_new'] = le_Info.fit_transform(DS.Info.values)

del DS["Source"]
del DS["Destination"]
del DS["Protocol"]
del DS["Info"]
DS['Flag'] = DS['ML']
del DS["ML"]

In [28]:
# Разбиваем датасет на обучающий набор данных и выборку валидации
DS_main = DS.loc[:int(len(DS)/2),:]
DS_valid = DS.loc[int(len(DS)/2):,:]
DS_valid.reset_index(drop= True , inplace= True )

In [29]:
# Выделение целевых переменных из массивов данных
main_X = DS_main.iloc[:,:-1]
main_Y = DS_main.iloc[:,-1]
valid_X = DS_valid.iloc[:,:-1]
valid_Y = DS_valid.iloc[:,-1]

In [30]:
# Разбиваем обучающий набор данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(main_X, main_Y, test_size=0.3, random_state=42)

In [31]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [32]:
print(f"Точность ML-модели при обучении: {clf.score(X_test, y_test)}")
print(f"Точность ML-модели на тестовой выборке: {clf.score(valid_X, valid_Y)}")

Точность ML-модели при обучении: 0.9999541515748934
Точность ML-модели на тестовой выборке: 0.9013144924280284


In [33]:
# Конвертируем прогноз модели в Pandas Series
attack_predict = pd.Series(clf.predict(valid_X))

In [34]:
# Обратное кодирование столбцов в текстовые значения
DS_valid['Source'] = le_Source.inverse_transform(DS_valid.Source_new.values)
DS_valid['Destination'] = le_Destination.inverse_transform(DS_valid.Destination_new.values)
DS_valid['Protocol'] = le_Protocol.inverse_transform(DS_valid.Protocol_new.values)
DS_valid['Info'] = le_Info.inverse_transform(DS_valid.Info_new.values)

del DS_valid["Source_new"]
del DS_valid["Destination_new"]
del DS_valid["Protocol_new"]
del DS_valid["Info_new"]

DS_valid['Class'] = DS_valid['Flag']
del DS_valid["Flag"]

In [35]:
# Добавляем столбец с результатаими работы ML-модели
DS_valid["Predict"] = attack_predict

In [36]:
print(f"Всего наблюдений в валидационном датасете: {len(DS_valid)}")
print(f"Всего размеченных атак: {DS_valid.Class.value_counts()[1]}")
print(f"Всего атак по версии ML-модели: {DS_valid.Predict.value_counts()[1]}")

Всего наблюдений в валидационном датасете: 218107
Всего размеченных атак: 149553
Всего атак по версии ML-модели: 145865


In [37]:
# Сравнение результатов работы модели с изначальной разметкой. Данная часть датасета визуализирует ddos-атаку
# Колонка Class - изначальная разметка (1 - атака, 0 - нормальный трафик), колонка Predict - результат работы ML-модели.
filt = DS_valid["Predict"] == 1
DS_valid[filt].head(30)

,No.,Time,Length,Source,Destination,Protocol,Info,Class,Predict
0,97604,287.320751,54,213.119.82.215,192.168.10.10,TCP,37460 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
1,97605,287.321648,54,228.34.114.48,192.168.10.10,TCP,52200 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
2,97606,287.322597,54,134.247.197.118,192.168.10.10,TCP,23618 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
3,97607,287.323768,54,9.12.158.91,192.168.10.10,TCP,28621 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
4,97608,287.324673,54,104.199.10.36,192.168.10.10,TCP,21553 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
5,97609,287.325674,54,114.182.15.192,192.168.10.10,TCP,5957 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
6,97610,287.326847,54,172.147.71.250,192.168.10.10,TCP,50570 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
7,97611,287.327977,54,220.232.240.4,192.168.10.10,TCP,7948 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
8,97612,287.328872,54,135.64.61.66,192.168.10.10,TCP,48955 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1
9,97613,287.330358,54,52.12.216.22,192.168.10.10,TCP,48911 > 80 [SYN] Seq=0 Win=8192 Len=0,1,1


In [38]:
# Сравнение результатов работы модели по отлову ICMP пакетов с изначальной разметкой
# Колонка Class - изначальная разметка (1 - атака, 0 - нормальный трафик), колонка Predict - результат работы ML-модели.
filt = DS_valid["Protocol"] == "ICMP"
DS_valid[filt].head(50)

,No.,Time,Length,Source,Destination,Protocol,Info,Class,Predict
10715,108319,310.187994,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10716,108320,320.190169,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10717,108321,330.192863,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10718,108322,340.195160,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10719,108323,350.198606,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10720,108324,360.201016,42,192.168.10.5,192.168.10.2,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10721,108325,370.203791,42,192.168.10.5,192.168.10.3,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10722,108326,380.207075,42,192.168.10.5,192.168.10.3,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10723,108327,390.209320,42,192.168.10.5,192.168.10.3,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1
10724,108328,400.211828,42,192.168.10.5,192.168.10.3,ICMP,"Echo (ping) request id=0x0000, seq=0/0, ttl=6...",1,1


In [39]:
# Сравнение результатов работы модели по отлову HTTP протокола с изначальной разметкой
# Колонка Class - изначальная разметка (1 - атака, 0 - нормальный трафик), колонка Predict - результат работы ML-модели.
filt = DS_valid["Protocol"] == "HTTP"
DS_valid[filt].head(50)

,No.,Time,Length,Source,Destination,Protocol,Info,Class,Predict
18300,2401,5.686226,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
18301,2402,5.686226,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
21110,5211,51.359821,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
21111,5212,51.359821,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
95803,2401,2.216304,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
95804,2402,2.216304,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
98613,5211,44.625846,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
98614,5212,44.625846,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
111940,3337,8.063204,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1
111941,3338,8.063204,242,192.168.10.5,216.58.209.238,HTTP,GET / HTTP/1.1,1,1


In [40]:
# Примеры ошибок модели
# Колонка Class - изначальная разметка (1 - атака, 0 - нормальный трафик), колонка Predict - результат работы ML-модели.
filt = (DS_valid["Predict"] == 1) & (DS_valid["Class"] == 0)
DS_valid[filt].sample(n=50).sort_index()

,No.,Time,Length,Source,Destination,Protocol,Info,Class,Predict
8452,106056,305.684984,54,192.168.10.2,192.168.10.8,TCP,56613 > 26150 [<None>] Seq=0 Win=777 Len=0,0,1
8539,106143,305.831626,125,192.168.10.5,192.168.10.10,TCP,34813 > 42925 [<None>] Seq=2882 Win=833 Len=71,0,1
8708,106312,306.158039,68,192.168.10.8,192.168.10.6,TCP,55923 > 19256 [<None>] Seq=1890 Win=764 Len=14,0,1
9155,106759,307.046402,54,192.168.10.5,192.168.10.10,TCP,3229 > 27125 [<None>] Seq=0 Win=454 Len=0,0,1
9289,106893,307.281695,99,192.168.10.2,192.168.10.5,TCP,4821 > 25393 [<None>] Seq=3520 Win=949 Len=45,0,1
9383,106987,307.562339,81,192.168.10.10,192.168.10.5,TCP,9059 > 7085 [<None>] Seq=3597 Win=628 Len=27,0,1
9516,107120,307.808831,54,192.168.10.5,192.168.10.6,TCP,44476 > 25314 [FIN] Seq=3335 Win=343 Len=0,0,1
10092,107696,308.943791,140,192.168.10.7,192.168.10.10,TCP,44253 > 12130 [<None>] Seq=4953 Win=400 Len=86,0,1
10198,107802,309.135205,126,192.168.10.3,192.168.10.4,TCP,25724 > 28885 [<None>] Seq=3749 Win=614 Len=72,0,1
10294,107898,309.420118,54,192.168.10.10,192.168.10.7,TCP,11152 > 46220 [<None>] Seq=2386 Win=362 Len=0,0,1
